In [158]:
import os
from string import punctuation
import datetime
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import random
%matplotlib inline

from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
import xgboost as xgb

In [159]:
TRAIN_DATA = os.path.join(os.getcwd(),'data','train.json')
TEST_DATA = os.path.join(os.getcwd(),'data','test.json')
LEAKAGE_FEATURE = os.path.join(os.getcwd(),'data','listing_image_time.csv')

train_df = pd.read_json(TRAIN_DATA)
test_df = pd.read_json(TEST_DATA)

leakage_df = pd.read_csv(LEAKAGE_FEATURE)
leakage_df.columns = ["listing_id", "time_stamp"]
# reassign the only one timestamp from April, all others from Oct/Nov
leakage_df.loc[80240,"time_stamp"] = 1478129766 

train_ids = train_df[['listing_id', 'interest_level']]
test_ids = test_df['listing_id']

In [160]:
train_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [161]:
def runXGB(train_X, train_y, test_X, test_y=None, seed_val=321, num_rounds=400):
    """ Creates and trains xgb classifier. Can be used for test and CV
    
    :param train_X: 
    :param train_y: 
    :param test_X: 
    :param test_y: 
    :param seed_val: 
    :param num_rounds: 
    :return: 
    """
    params = {'objective': 'multi:softprob',
              'eta': 0.02,
              'max_depth': 6,
              'silent': 1,
              'num_class': 3,
              'eval_metric': "mlogloss",
              'min_child_weight': 1,
              'subsample': 0.7,
              'colsample_bytree': 0.7,
              'seed': seed_val
              }

    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        evals = [(xgtrain, 'train'), (xgtest, 'test')]
        model = xgb.train(params=params, dtrain=xgtrain, num_boost_round=num_rounds,
                          evals=evals, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(params=params, dtrain=xgtrain, num_boost_round=num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [162]:
def manager_interest_level(train_df1, test_df1):
    index=list(range(train_df1.shape[0]))
    random.shuffle(index)
    a=[np.nan]*len(train_df1)
    b=[np.nan]*len(train_df1)
    c=[np.nan]*len(train_df1)

    for i in range(5):
        building_level={}
        for j in train_df1['manager_id'].values:
            building_level[j]=[0,0,0]
        test_index=index[int((i*train_df1.shape[0])/5):int(((i+1)*train_df1.shape[0])/5)]
        train_index=list(set(index).difference(test_index))
        for j in train_index:
            temp=train_df1.iloc[j]
            if temp['interest_level']=='low':
                building_level[temp['manager_id']][0]+=1
            if temp['interest_level']=='medium':
                building_level[temp['manager_id']][1]+=1
            if temp['interest_level']=='high':
                building_level[temp['manager_id']][2]+=1
        for j in test_index:
            temp=train_df1.iloc[j]
            if sum(building_level[temp['manager_id']])!=0:
                a[j]=building_level[temp['manager_id']][0]*1.0/sum(building_level[temp['manager_id']])
                b[j]=building_level[temp['manager_id']][1]*1.0/sum(building_level[temp['manager_id']])
                c[j]=building_level[temp['manager_id']][2]*1.0/sum(building_level[temp['manager_id']])
    train_df1['manager_level_low']=a
    train_df1['manager_level_medium']=b
    train_df1['manager_level_high']=c



    a=[]
    b=[]
    c=[]
    building_level={}
    for j in train_df1['manager_id'].values:
        building_level[j]=[0,0,0]
    for j in range(train_df1.shape[0]):
        temp=train_df1.iloc[j]
        if temp['interest_level']=='low':
            building_level[temp['manager_id']][0]+=1
        if temp['interest_level']=='medium':
            building_level[temp['manager_id']][1]+=1
        if temp['interest_level']=='high':
            building_level[temp['manager_id']][2]+=1

    for i in test_df1['manager_id'].values:
        if i not in building_level.keys():
            a.append(np.nan)
            b.append(np.nan)
            c.append(np.nan)
        else:
            a.append(building_level[i][0]*1.0/sum(building_level[i]))
            b.append(building_level[i][1]*1.0/sum(building_level[i]))
            c.append(building_level[i][2]*1.0/sum(building_level[i]))
    test_df1['manager_level_low']=a
    test_df1['manager_level_medium']=b
    test_df1['manager_level_high']=c
    return train_df1, test_df1

In [163]:
def building_interest_level(train_df1, test_df1):
    index=list(range(train_df1.shape[0]))
    random.shuffle(index)
    a=[np.nan]*len(train_df1)
    b=[np.nan]*len(train_df1)
    c=[np.nan]*len(train_df1)

    for i in range(5):
        building_level={}
        for j in train_df1['building_id'].values:
            building_level[j]=[0,0,0]
        test_index=index[int((i*train_df1.shape[0])/5):int(((i+1)*train_df1.shape[0])/5)]
        train_index=list(set(index).difference(test_index))
        for j in train_index:
            temp=train_df1.iloc[j]
            if temp['interest_level']=='low':
                building_level[temp['building_id']][0]+=1
            if temp['interest_level']=='medium':
                building_level[temp['building_id']][1]+=1
            if temp['interest_level']=='high':
                building_level[temp['building_id']][2]+=1
        for j in test_index:
            temp=train_df1.iloc[j]
            if sum(building_level[temp['building_id']])!=0:
                a[j]=building_level[temp['building_id']][0]*1.0/sum(building_level[temp['building_id']])
                b[j]=building_level[temp['building_id']][1]*1.0/sum(building_level[temp['building_id']])
                c[j]=building_level[temp['building_id']][2]*1.0/sum(building_level[temp['building_id']])
    train_df1['building_level_low']=a
    train_df1['building_level_medium']=b
    train_df1['building_level_high']=c



    a=[]
    b=[]
    c=[]
    building_level={}
    for j in train_df1['building_id'].values:
        building_level[j]=[0,0,0]
    for j in range(train_df1.shape[0]):
        temp=train_df1.iloc[j]
        if temp['interest_level']=='low':
            building_level[temp['building_id']][0]+=1
        if temp['interest_level']=='medium':
            building_level[temp['building_id']][1]+=1
        if temp['interest_level']=='high':
            building_level[temp['building_id']][2]+=1

    for i in test_df1['building_id'].values:
        if i not in building_level.keys():
            a.append(np.nan)
            b.append(np.nan)
            c.append(np.nan)
        else:
            a.append(building_level[i][0]*1.0/sum(building_level[i]))
            b.append(building_level[i][1]*1.0/sum(building_level[i]))
            c.append(building_level[i][2]*1.0/sum(building_level[i]))
    test_df1['building_level_low']=a
    test_df1['building_level_medium']=b
    test_df1['building_level_high']=c
    return train_df1, test_df1

In [164]:
def data_concat(df1, df2):
    df = pd.concat([df1,df2], ignore_index=True)
    df = df.drop(labels=['interest_level'], axis=1)
    return df.reset_index()

In [165]:
def merge_with_img_df(df, image_date):   
    image_date["img_date"] = pd.to_datetime(image_date["time_stamp"], unit="s")
    image_date["img_days_passed"] = (image_date["img_date"].max() - image_date["img_date"]).astype("timedelta64[D]").astype(int)
    image_date["img_date_month"] = image_date["img_date"].dt.month
    image_date["img_date_week"] = image_date["img_date"].dt.week
    image_date["img_date_day"] = image_date["img_date"].dt.day
    image_date["img_date_dayofweek"] = image_date["img_date"].dt.dayofweek
    image_date["img_date_dayofyear"] = image_date["img_date"].dt.dayofyear
    image_date["img_date_hour"] = image_date["img_date"].dt.hour
    image_date["img_date_monthBeginMidEnd"] = image_date["img_date_day"].apply(lambda x: 1 if x<10 else 2 if x<20 else 3)
    
    df = pd.merge(df, image_date, on="listing_id", how="left")
    return df

In [166]:
def data_split(df):
    target_num_map = {'high':0, 'medium':1, 'low':2}
    train_df1 = df[ df['listing_id'].isin(train_ids['listing_id'].values) ]
    train_df1['interest_level'] = train_df1['listing_id'].apply(
                              lambda x: train_ids.loc[train_ids['listing_id'] == x, 'interest_level'].values[0])
    train_df1['interest_level'] = train_df1['interest_level'].apply(lambda x: target_num_map[x])
    
    test_df1 = df[ df['listing_id'].isin(test_ids.values) ]

    return train_df1, test_df1

In [167]:
def new_features_df(df):
    
    remove_punct_map = dict.fromkeys(map(ord, punctuation))
    
    # price and rooms additional features
    df["logprice"] = np.log(df["price"])
    df["price_t"] =df["price"]/df["bedrooms"]
    df["room_sum"] = df["bedrooms"]+df["bathrooms"] 
    df['price_per_room'] = df['price']/df['room_sum']
    
    # location feature
    df["pos"] = df['longitude'].round(3).astype(str) + '_' + df['latitude'].round(3).astype(str)
    vals = df['pos'].value_counts()
    dvals = vals.to_dict()
    df["density"] = df['pos'].apply(lambda x: dvals.get(x, vals.min()))
    df = df.drop(labels=['pos'], axis=1)
    
    
    # deal with *created* feature
    df["created"] = pd.to_datetime(df["created"])
    df["created_year"] = df["created"].dt.year
    df["created_month"] = df["created"].dt.month
    df["created_day"] = df["created"].dt.day
    df["created_hour"] = df["created"].dt.hour
      
    # deal with *photos* feature
    # nothing else we can do except count photos
    df["num_photos"] = df["photos"].apply(len)
    
    # deal with *features* feature
    # TODO: tokenize or similar
    df["num_features"] = df["features"].apply(len)
    
    # deal with *description* feature
    # TODO: try word2vec representation or similar
    df['desc'] = df['description']
    df['desc'] = df['desc'].apply(lambda x: x.replace('<p><a  website_redacted ', ''))
    df['desc'] = df['desc'].apply(lambda x: x.replace('!<br /><br />', ''))
    df['desc'] = df['desc'].apply(lambda x: x.replace('!<br /><br />', ''))
    df['desc'] = df['desc'].apply(lambda x: x.translate(remove_punct_map))
    df["num_description_words"] = df["desc"].apply(lambda x: len(list(filter(None, x.split(" ")))))
    
    # deal with *description* feature
    # if present == 1, else == 0
    df['building_id_present'] = df['building_id'].apply(lambda x: 1 if x != '0' else 0)
    
    manager_count = df.groupby('manager_id').count().iloc[:,-1].to_dict()
    df['manager_count'] = df['manager_id'].apply(lambda x: manager_count.get(x, 0))
    
    building_count = df.groupby('building_id').count().iloc[:,-1].to_dict()
    building_count.pop('0')
    df['building_count'] = df['building_id'].apply(lambda x: building_count.get(x, 1))
    
    return df

In [168]:
# counter = {}
# for ind, row in df.iterrows():
#     for feature in row['features']:
#         counter[feature] = counter.get(feature, 0) + 1

def add_desc_features(df):
    general = ['elevator', 'hardwood', 'doorman', 'dishwasher', 'no fee', 'reduced fee' 'laundry', 'fitness', 'cat', 'dog', 
               'roof', 'outdoor space', 'dining', 'internet', 'balcon', 'pool', 'new construction', 'terrace',
               'exclusive', 'loft', 'garden', 'wheelchair', 'fireplace', 'garage', 'furnished', 'multi-level']
    pre_war = ['prewar', 'pre-war', 'pre war']
    for item in general:
        df['feature'+'_'+item] = df['features'].apply(lambda x: 1 if item in " ".join(x).lower() else 0)
    df['feature_prewar'] = df['features'].apply(lambda x: 1 if any(item in " ".join(x).lower() for item in pre_war) else 0)
    return df

In [169]:
def remove_categorical_features(df):
    categogical = ['created', 'description', 'photos', 'street_address', 'features', 'desc', 'img_date']
    df = df.drop(labels=categogical, axis=1)
    return df

In [170]:
def encode_categorical_features(df):
    # categorical = ['building_id', 'display_address', 'manager_id']
    categorical = [c for c in df.columns if df[c].dtype.name == 'object']
    for f in categorical:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df[f].values))
        df[f] = lbl.transform(list(df[f].values))
    return df

In [171]:
train_df, test_df = manager_interest_level(train_df, test_df)
train_df, test_df = building_interest_level(train_df, test_df)

df = data_concat(train_df, test_df)
df = merge_with_img_df(df, leakage_df)
print('train_df:', train_df.shape)
print('test_df:', test_df.shape)
print('df:', df.shape)

train_df: (49352, 21)
test_df: (74659, 20)
df: (124011, 31)


In [172]:
df.head()

,index,bathrooms,bedrooms,building_id,building_level_high,building_level_low,building_level_medium,created,description,display_address,...,time_stamp,img_date,img_days_passed,img_date_month,img_date_week,img_date_day,img_date_dayofweek,img_date_dayofyear,img_date_hour,img_date_monthBeginMidEnd
0,0,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,0.000000,1.000000,0.000000,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,...,1478091590,2016-11-02 12:59:50,19,11,44,2,2,307,12,1
1,1,1.0,2,c5c8a357cba207596b04d1afd1e4f130,0.000000,0.882353,0.117647,2016-06-12 12:19:27,,Columbus Avenue,...,1478129766,2016-11-02 23:36:06,19,11,44,2,2,307,23,1
2,2,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,0.177778,0.400000,0.422222,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,...,1478714436,2016-11-09 18:00:36,12,11,45,9,2,314,18,1
3,3,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,0.013158,0.894737,0.092105,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,...,1478714444,2016-11-09 18:00:44,12,11,45,9,2,314,18,1
4,4,1.0,4,0,0.023217,0.916478,0.060305,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,...,1478714464,2016-11-09 18:01:04,12,11,45,9,2,314,18,1


In [173]:
print('DATA VARIABLES\n')

print('='*30)
categorical_columns = [c for c in df.columns if df[c].dtype.name == 'object']
numerical_columns   = [c for c in df.columns if df[c].dtype.name != 'object']
print('categogical columns:')
print(categorical_columns)
print('='*30)
print('numerical columns:')
print(numerical_columns)

DATA VARIABLES

categogical columns:
['building_id', 'created', 'description', 'display_address', 'features', 'manager_id', 'photos', 'street_address']
numerical columns:
['index', 'bathrooms', 'bedrooms', 'building_level_high', 'building_level_low', 'building_level_medium', 'latitude', 'listing_id', 'longitude', 'manager_level_high', 'manager_level_low', 'manager_level_medium', 'price', 'time_stamp', 'img_date', 'img_days_passed', 'img_date_month', 'img_date_week', 'img_date_day', 'img_date_dayofweek', 'img_date_dayofyear', 'img_date_hour', 'img_date_monthBeginMidEnd']


In [174]:
df = new_features_df(df)

In [175]:
df = add_desc_features(df)

In [176]:
df.head()

,index,bathrooms,bedrooms,building_id,building_level_high,building_level_low,building_level_medium,created,description,display_address,...,feature_terrace,feature_exclusive,feature_loft,feature_garden,feature_wheelchair,feature_fireplace,feature_garage,feature_furnished,feature_multi-level,feature_prewar
0,0,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,0.000000,1.000000,0.000000,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,...,0,0,0,0,0,0,0,0,0,0
1,1,1.0,2,c5c8a357cba207596b04d1afd1e4f130,0.000000,0.882353,0.117647,2016-06-12 12:19:27,,Columbus Avenue,...,0,0,0,0,0,0,0,0,0,0
2,2,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,0.177778,0.400000,0.422222,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,...,0,0,0,0,0,0,0,0,0,0
3,3,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,0.013158,0.894737,0.092105,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,...,0,0,0,0,0,0,0,0,0,0
4,4,1.0,4,0,0.023217,0.916478,0.060305,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,...,0,0,0,0,0,0,0,0,0,1


In [177]:
df = remove_categorical_features(df)

In [178]:
categorical_columns = [c for c in df.columns if df[c].dtype.name == 'object']
numerical_columns   = [c for c in df.columns if df[c].dtype.name != 'object']

print('DATA VARIABLES\n')
print('='*30)
print('categogical columns:')
print()
print('\n'.join(categorical_columns))
print('='*30)
print('numerical columns:')
print()
print('\n'.join(numerical_columns))
print('='*30)

DATA VARIABLES

categogical columns:

building_id
display_address
manager_id
numerical columns:

index
bathrooms
bedrooms
building_level_high
building_level_low
building_level_medium
latitude
listing_id
longitude
manager_level_high
manager_level_low
manager_level_medium
price
time_stamp
img_days_passed
img_date_month
img_date_week
img_date_day
img_date_dayofweek
img_date_dayofyear
img_date_hour
img_date_monthBeginMidEnd
logprice
price_t
room_sum
price_per_room
density
created_year
created_month
created_day
created_hour
num_photos
num_features
num_description_words
building_id_present
manager_count
building_count
feature_elevator
feature_hardwood
feature_doorman
feature_dishwasher
feature_no fee
feature_reduced feelaundry
feature_fitness
feature_cat
feature_dog
feature_roof
feature_outdoor space
feature_dining
feature_internet
feature_balcon
feature_pool
feature_new construction
feature_terrace
feature_exclusive
feature_loft
feature_garden
feature_wheelchair
feature_fireplace
feature_ga

In [179]:
df = encode_categorical_features(df)

In [180]:
print('DATA VARIABLES\n')

print('='*30)
categorical_columns = [c for c in df.columns if df[c].dtype.name == 'object']
numerical_columns   = [c for c in df.columns if df[c].dtype.name != 'object']
print('categogical columns:')
print(categorical_columns)
print('='*30)
print('numerical columns:')
print(numerical_columns)

DATA VARIABLES

categogical columns:
[]
numerical columns:
['index', 'bathrooms', 'bedrooms', 'building_id', 'building_level_high', 'building_level_low', 'building_level_medium', 'display_address', 'latitude', 'listing_id', 'longitude', 'manager_id', 'manager_level_high', 'manager_level_low', 'manager_level_medium', 'price', 'time_stamp', 'img_days_passed', 'img_date_month', 'img_date_week', 'img_date_day', 'img_date_dayofweek', 'img_date_dayofyear', 'img_date_hour', 'img_date_monthBeginMidEnd', 'logprice', 'price_t', 'room_sum', 'price_per_room', 'density', 'created_year', 'created_month', 'created_day', 'created_hour', 'num_photos', 'num_features', 'num_description_words', 'building_id_present', 'manager_count', 'building_count', 'feature_elevator', 'feature_hardwood', 'feature_doorman', 'feature_dishwasher', 'feature_no fee', 'feature_reduced feelaundry', 'feature_fitness', 'feature_cat', 'feature_dog', 'feature_roof', 'feature_outdoor space', 'feature_dining', 'feature_internet', '

In [181]:
train_df_new, test_df_new = data_split(df)

/home/dima/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/dima/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [182]:
train_ids_new = train_df['listing_id']
test_ids_new = test_df['listing_id']

In [183]:
X = train_df_new.drop(labels=['listing_id','interest_level'], axis=1)
y = train_df_new['interest_level']

In [184]:
y.shape

(49352,)

In [185]:
from sklearn.model_selection import StratifiedKFold, KFold
cv_scores = []
models = []
skf = StratifiedKFold(n_splits=4, shuffle=True)

for train_index, test_index in skf.split(X, y):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    preds, model = runXGB(X_train, y_train, X_test, y_test)
    cv_scores.append(log_loss(y_test, preds))
    models.append(model)
    #print(cv_scores)
    #break


TRAIN: 37013 TEST: 12339
[0]	train-mlogloss:1.0841	test-mlogloss:1.08468
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.07015	test-mlogloss:1.07124
[2]	train-mlogloss:1.05658	test-mlogloss:1.05823
[3]	train-mlogloss:1.04347	test-mlogloss:1.04561
[4]	train-mlogloss:1.03098	test-mlogloss:1.03379
[5]	train-mlogloss:1.01887	test-mlogloss:1.02223
[6]	train-mlogloss:1.00696	test-mlogloss:1.01085
[7]	train-mlogloss:0.995636	test-mlogloss:1.00005
[8]	train-mlogloss:0.984416	test-mlogloss:0.989421
[9]	train-mlogloss:0.973593	test-mlogloss:0.979135
[10]	train-mlogloss:0.963117	test-mlogloss:0.969186
[11]	train-mlogloss:0.952946	test-mlogloss:0.959515
[12]	train-mlogloss:0.94298	test-mlogloss:0.950102
[13]	train-mlogloss:0.93348	test-mlogloss:0.940971
[14]	train-mlogloss:0.92406	test-mlogloss:0.932131
[15]	train-mlogloss:0.914934	test-mlogloss:0.923494
[16]	train-mlogloss:0

[155]	train-mlogloss:0.536892	test-mlogloss:0.595088
[156]	train-mlogloss:0.536191	test-mlogloss:0.594584
[157]	train-mlogloss:0.535474	test-mlogloss:0.594193
[158]	train-mlogloss:0.534814	test-mlogloss:0.593797
[159]	train-mlogloss:0.534167	test-mlogloss:0.593359
[160]	train-mlogloss:0.533567	test-mlogloss:0.592994
[161]	train-mlogloss:0.532928	test-mlogloss:0.59254
[162]	train-mlogloss:0.532329	test-mlogloss:0.592155
[163]	train-mlogloss:0.531733	test-mlogloss:0.591787
[164]	train-mlogloss:0.531096	test-mlogloss:0.59135
[165]	train-mlogloss:0.530477	test-mlogloss:0.590918
[166]	train-mlogloss:0.529864	test-mlogloss:0.590543
[167]	train-mlogloss:0.529294	test-mlogloss:0.590175
[168]	train-mlogloss:0.52868	test-mlogloss:0.589722
[169]	train-mlogloss:0.528091	test-mlogloss:0.58943
[170]	train-mlogloss:0.527505	test-mlogloss:0.589105
[171]	train-mlogloss:0.52692	test-mlogloss:0.588779
[172]	train-mlogloss:0.526318	test-mlogloss:0.588449
[173]	train-mlogloss:0.525772	test-mlogloss:0.58821

[311]	train-mlogloss:0.473221	test-mlogloss:0.56334
[312]	train-mlogloss:0.472881	test-mlogloss:0.563265
[313]	train-mlogloss:0.472634	test-mlogloss:0.563189
[314]	train-mlogloss:0.472363	test-mlogloss:0.563077
[315]	train-mlogloss:0.472084	test-mlogloss:0.562954
[316]	train-mlogloss:0.471718	test-mlogloss:0.562857
[317]	train-mlogloss:0.471466	test-mlogloss:0.562889
[318]	train-mlogloss:0.471215	test-mlogloss:0.562881
[319]	train-mlogloss:0.47093	test-mlogloss:0.562737
[320]	train-mlogloss:0.470604	test-mlogloss:0.562623
[321]	train-mlogloss:0.470373	test-mlogloss:0.562499
[322]	train-mlogloss:0.470097	test-mlogloss:0.562417
[323]	train-mlogloss:0.46982	test-mlogloss:0.562346
[324]	train-mlogloss:0.469554	test-mlogloss:0.56223
[325]	train-mlogloss:0.469278	test-mlogloss:0.562298
[326]	train-mlogloss:0.468975	test-mlogloss:0.562216
[327]	train-mlogloss:0.468732	test-mlogloss:0.562167
[328]	train-mlogloss:0.468455	test-mlogloss:0.562158
[329]	train-mlogloss:0.4682	test-mlogloss:0.562043

[92]	train-mlogloss:0.612779	test-mlogloss:0.62278
[93]	train-mlogloss:0.611184	test-mlogloss:0.621302
[94]	train-mlogloss:0.60962	test-mlogloss:0.619827
[95]	train-mlogloss:0.608108	test-mlogloss:0.618405
[96]	train-mlogloss:0.60654	test-mlogloss:0.617007
[97]	train-mlogloss:0.605032	test-mlogloss:0.615686
[98]	train-mlogloss:0.603632	test-mlogloss:0.614346
[99]	train-mlogloss:0.602184	test-mlogloss:0.613032
[100]	train-mlogloss:0.600734	test-mlogloss:0.611772
[101]	train-mlogloss:0.599341	test-mlogloss:0.610556
[102]	train-mlogloss:0.597879	test-mlogloss:0.609305
[103]	train-mlogloss:0.596551	test-mlogloss:0.607883
[104]	train-mlogloss:0.595289	test-mlogloss:0.606698
[105]	train-mlogloss:0.594009	test-mlogloss:0.605485
[106]	train-mlogloss:0.592717	test-mlogloss:0.604317
[107]	train-mlogloss:0.591461	test-mlogloss:0.603136
[108]	train-mlogloss:0.59029	test-mlogloss:0.601983
[109]	train-mlogloss:0.589084	test-mlogloss:0.600879
[110]	train-mlogloss:0.587917	test-mlogloss:0.599834
[111]

[248]	train-mlogloss:0.50365	test-mlogloss:0.539398
[249]	train-mlogloss:0.503306	test-mlogloss:0.539411
[250]	train-mlogloss:0.502942	test-mlogloss:0.539256
[251]	train-mlogloss:0.502585	test-mlogloss:0.539225
[252]	train-mlogloss:0.502192	test-mlogloss:0.539004
[253]	train-mlogloss:0.501818	test-mlogloss:0.538857
[254]	train-mlogloss:0.501421	test-mlogloss:0.538712
[255]	train-mlogloss:0.501086	test-mlogloss:0.538577
[256]	train-mlogloss:0.500727	test-mlogloss:0.538404
[257]	train-mlogloss:0.500451	test-mlogloss:0.538268
[258]	train-mlogloss:0.500076	test-mlogloss:0.538278
[259]	train-mlogloss:0.499688	test-mlogloss:0.538064
[260]	train-mlogloss:0.499297	test-mlogloss:0.537907
[261]	train-mlogloss:0.498978	test-mlogloss:0.537715
[262]	train-mlogloss:0.498626	test-mlogloss:0.537723
[263]	train-mlogloss:0.498207	test-mlogloss:0.537553
[264]	train-mlogloss:0.497833	test-mlogloss:0.537388
[265]	train-mlogloss:0.497526	test-mlogloss:0.537248
[266]	train-mlogloss:0.497195	test-mlogloss:0.5

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.07078	test-mlogloss:1.07083
[2]	train-mlogloss:1.05758	test-mlogloss:1.05771
[3]	train-mlogloss:1.04481	test-mlogloss:1.045
[4]	train-mlogloss:1.0324	test-mlogloss:1.03271
[5]	train-mlogloss:1.0203	test-mlogloss:1.02069
[6]	train-mlogloss:1.00871	test-mlogloss:1.0092
[7]	train-mlogloss:0.997389	test-mlogloss:0.997947
[8]	train-mlogloss:0.986397	test-mlogloss:0.987023
[9]	train-mlogloss:0.975797	test-mlogloss:0.97651
[10]	train-mlogloss:0.96561	test-mlogloss:0.966416
[11]	train-mlogloss:0.95551	test-mlogloss:0.956406
[12]	train-mlogloss:0.945785	test-mlogloss:0.94688
[13]	train-mlogloss:0.936327	test-mlogloss:0.937524
[14]	train-mlogloss:0.927106	test-mlogloss:0.928322
[15]	train-mlogloss:0.918105	test-mlogloss:0.919391
[16]	train-mlogloss:0.909414	test-mlogloss:0.910782
[17]	train-mlogloss:0.901001	test-mlogloss:0.902533
[18]	train-mlogloss:0.892754	test-mlogloss:0.894412
[19]	train-mlogloss:0.884745	test

[158]	train-mlogloss:0.542851	test-mlogloss:0.570505
[159]	train-mlogloss:0.542184	test-mlogloss:0.570323
[160]	train-mlogloss:0.541569	test-mlogloss:0.5699
[161]	train-mlogloss:0.54098	test-mlogloss:0.569665
[162]	train-mlogloss:0.540407	test-mlogloss:0.569192
[163]	train-mlogloss:0.539805	test-mlogloss:0.568744
[164]	train-mlogloss:0.539195	test-mlogloss:0.568238
[165]	train-mlogloss:0.538593	test-mlogloss:0.567819
[166]	train-mlogloss:0.53796	test-mlogloss:0.567332
[167]	train-mlogloss:0.537371	test-mlogloss:0.56692
[168]	train-mlogloss:0.536758	test-mlogloss:0.566534
[169]	train-mlogloss:0.53619	test-mlogloss:0.566201
[170]	train-mlogloss:0.535636	test-mlogloss:0.565814
[171]	train-mlogloss:0.535101	test-mlogloss:0.565464
[172]	train-mlogloss:0.53453	test-mlogloss:0.565072
[173]	train-mlogloss:0.533969	test-mlogloss:0.564895
[174]	train-mlogloss:0.533433	test-mlogloss:0.564543
[175]	train-mlogloss:0.532837	test-mlogloss:0.564171
[176]	train-mlogloss:0.532281	test-mlogloss:0.563778


[314]	train-mlogloss:0.479488	test-mlogloss:0.541865
[315]	train-mlogloss:0.479216	test-mlogloss:0.541761
[316]	train-mlogloss:0.47889	test-mlogloss:0.541684
[317]	train-mlogloss:0.47858	test-mlogloss:0.542002
[318]	train-mlogloss:0.478286	test-mlogloss:0.541888
[319]	train-mlogloss:0.478039	test-mlogloss:0.541769
[320]	train-mlogloss:0.477677	test-mlogloss:0.541745
[321]	train-mlogloss:0.477405	test-mlogloss:0.542052
[322]	train-mlogloss:0.477101	test-mlogloss:0.541975
[323]	train-mlogloss:0.476906	test-mlogloss:0.5419
[324]	train-mlogloss:0.476638	test-mlogloss:0.541742
[325]	train-mlogloss:0.476329	test-mlogloss:0.541608
[326]	train-mlogloss:0.47604	test-mlogloss:0.541449
[327]	train-mlogloss:0.475768	test-mlogloss:0.541343
[328]	train-mlogloss:0.475467	test-mlogloss:0.541209
[329]	train-mlogloss:0.475179	test-mlogloss:0.541155
[330]	train-mlogloss:0.474855	test-mlogloss:0.541066
[331]	train-mlogloss:0.474539	test-mlogloss:0.540929
[332]	train-mlogloss:0.474308	test-mlogloss:0.54095

[68]	train-mlogloss:0.654229	test-mlogloss:0.694481
[69]	train-mlogloss:0.651711	test-mlogloss:0.692116
[70]	train-mlogloss:0.649199	test-mlogloss:0.689768
[71]	train-mlogloss:0.646795	test-mlogloss:0.687482
[72]	train-mlogloss:0.644322	test-mlogloss:0.685448
[73]	train-mlogloss:0.641986	test-mlogloss:0.683394
[74]	train-mlogloss:0.639678	test-mlogloss:0.681614
[75]	train-mlogloss:0.637457	test-mlogloss:0.679527
[76]	train-mlogloss:0.635227	test-mlogloss:0.677873
[77]	train-mlogloss:0.633004	test-mlogloss:0.676127
[78]	train-mlogloss:0.630838	test-mlogloss:0.674296
[79]	train-mlogloss:0.628756	test-mlogloss:0.672298
[80]	train-mlogloss:0.626698	test-mlogloss:0.67045
[81]	train-mlogloss:0.624641	test-mlogloss:0.668495
[82]	train-mlogloss:0.622658	test-mlogloss:0.667045
[83]	train-mlogloss:0.620655	test-mlogloss:0.665124
[84]	train-mlogloss:0.618722	test-mlogloss:0.663392
[85]	train-mlogloss:0.616807	test-mlogloss:0.661597
[86]	train-mlogloss:0.614977	test-mlogloss:0.65991
[87]	train-mlo

[224]	train-mlogloss:0.502139	test-mlogloss:0.576095
[225]	train-mlogloss:0.501738	test-mlogloss:0.575824
[226]	train-mlogloss:0.501388	test-mlogloss:0.575898
[227]	train-mlogloss:0.501004	test-mlogloss:0.575633
[228]	train-mlogloss:0.500532	test-mlogloss:0.575376
[229]	train-mlogloss:0.50014	test-mlogloss:0.575083
[230]	train-mlogloss:0.499788	test-mlogloss:0.574848
[231]	train-mlogloss:0.499321	test-mlogloss:0.5746
[232]	train-mlogloss:0.498985	test-mlogloss:0.57456
[233]	train-mlogloss:0.498625	test-mlogloss:0.574316
[234]	train-mlogloss:0.498243	test-mlogloss:0.574328
[235]	train-mlogloss:0.497868	test-mlogloss:0.574143
[236]	train-mlogloss:0.497427	test-mlogloss:0.57391
[237]	train-mlogloss:0.497037	test-mlogloss:0.574036
[238]	train-mlogloss:0.496649	test-mlogloss:0.574297
[239]	train-mlogloss:0.496236	test-mlogloss:0.574037
[240]	train-mlogloss:0.495914	test-mlogloss:0.573977
[241]	train-mlogloss:0.495449	test-mlogloss:0.573758
[242]	train-mlogloss:0.49508	test-mlogloss:0.573561

In [186]:
Xgtest = xgb.DMatrix(test_df_new.drop(labels=['listing_id'], axis=1))

In [187]:
def merge_several_folds_mean(data, nfolds):
    a = np.array(data[0])
    for i in range(1, nfolds):
        a += np.array(data[i])
    a /= nfolds
    return a.tolist()

In [188]:
def create_submission(predictions, test_id):
    result1 = pd.DataFrame(predictions, columns=["high", "medium", "low"])
    result1['listing_id'] = test_id.values
    now = datetime.datetime.now()
    sub_file = 'submission_' + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    result1.to_csv(sub_file, index=False)

In [194]:
np.mean(cv_scores)

0.55000254684179173

In [190]:
num_fold = 0
yfull_test = []
nfolds = len(models)

for i in range(nfolds):
    model = models[i]
    num_fold += 1
    print('Start KFold number {} from {}'.format(num_fold, nfolds))
    test_prediction = model.predict(Xgtest)
    yfull_test.append(test_prediction)

Start KFold number 1 from 4
Start KFold number 2 from 4
Start KFold number 3 from 4
Start KFold number 4 from 4


In [191]:
test_res = merge_several_folds_mean(yfull_test, nfolds)

In [192]:
create_submission(test_res, test_ids_new)